## Conversion of a dataframe with pressure measurements from SWDE experiment into a dataframe compatible with the efavor format

#### T. Janus
#### 20-11-2024

### Assumptions
* Single inlet with name given in variable `inlet_name` - default == "A"

### Short description


In [ ]:
# Library import
import pathlib
import pandas as pd
# Constants and global variables
path_to_datafile = pathlib.Path("efavor/swde/swde_test.xlsx")
inlet_name: str = "A"

In [ ]:
# Load and process the original dataframe
swde_data = pd.read_excel(path_to_datafile)\
    .drop(columns=["Device #"])\
    .rename(columns={"_node": "node"})\ 
    .set_index("Time")\
    .loc[lambda x: ~x.index.isna()]\
    .loc[lambda x: x.index.time >= pd.to_datetime("01:00:00").time()]\ # Times < 1am are disregarder
    .groupby("node")\
    .resample('15T')\
    .mean()\
    .reset_index()\
    .pivot(index='Time', columns='node', values='Pressure')

In [ ]:
# Modify the look of the dataframe to fit the efavor formatting convention
logger_name_map = {column: ix for ix, column in enumerate(swde_data.columns)}
# Creat an efavor-compatible dataframe
swde_data_efavor = swde_data\
    .assign(Inlet=lambda x: "A")\
    .set_index("Inlet")\
    .rename(columns=logger_name_map)

In [ ]:
swde_data_efavor.head(10)